In [14]:
#from datetime import datetime
#from dateutil.relativedelta import relativedelta
#import os

import json
import pandas as pd
import requests

In [20]:
# get the headers of the taxi data
url = 'https://futar.bkk.hu/api/query/v1/ws/otp/api/where/bicycle-rental.json?key=bd29005b-80cb-499b-968b-c7be1e6b3d37&version=3&appVersion=1&includeReferences=false'

response = requests.get(url)
data = response.json()

with open('mydata.json', 'w') as file:
    json.dump(data, file, indent=4)
    
with open('mydata2.json', 'w') as file:
    json.dump(data['data'], file, indent=4)

The structure of the data Dict:

    - "currentTime": time in Unix timestamp format (milliseconds since the Unix Epoch)
    - "version": 3,
    - "status": "OK",
    - "code": 200,
    - "text": "OK",
    - "data": all the information embedded here

The "data" key contains the following subkeys:

    - "list": the detailed information about the bikes and stations
    - "outOfRange": false    ?
    - "limitExceeded": false ?
    - "class": "listWithReferences"  ?


"list" is a list of dictionaries about bikes or bike stations

An example:
```json
        {
            "id": "42711896",
            "lat": 47.51032,
            "lon": 19.028615,
            "name": "Millen\u00e1ris",
            "code": "0213",
            "type": "stele",
            "bikes": 1,
            "spaces": 999,
            "style": {
                "icon": {
                    "name": "vehicle-rental/mol-bubi"
                }
            }
        }
```

Based on the [API documentation](https://bkkfutar.docs.apiary.io/#reference/0/bicyclerental/bicyclerental):

    - id (string):    MOL Bubi állomás azonosítója, e.g. 251962
    - lat (number):   latitude, e.g. 47.511182000000005
    - lon (number):   longitude, e.g. 19.080324700000002
    - name(string):   elnevezés, e.g. 'Dózsa György út'
    - code (string):  kód, e.g. '0612' (I guess this is the unique identitier of the station/bike)
    - type (enum):    kijelző típusa, e.g. '10inch' or 'character' (these denoted as strings)
    - bikes(number):  elérhető kerékpárok száma, e.g. 5
    - spaces (number): szabad kerékpár támaszok száma. Ha 0, akkor is elhelyezhető kerékpár az állomáson a "pót" támasznál. e.g. 10

In [22]:
# create a dataframe from the detailed data
bicycle_rental = pd.DataFrame(data['data']['list'])
bicycle_rental.head(10)

,id,lat,lon,name,code,type,bikes,spaces,style
96,42990629,47.492537,19.056618,Ferenciek tere,0516,stele,5,999,{'icon': {'name': 'vehicle-rental/mol-bubi'}}
72,42990719,47.509867,19.026453,Millenáris park - Fény utca,0208,stele,7,999,{'icon': {'name': 'vehicle-rental/mol-bubi'}}
19,42990701,47.506461,19.060056,Jókai utca - Zichy Jenő utca,0608,stele,6,999,{'icon': {'name': 'vehicle-rental/mol-bubi'}}
192,42990655,47.494215,19.060351,Astoria,0802,stele,9,999,{'icon': {'name': 'vehicle-rental/mol-bubi'}}
69,42990711,47.512197,19.038255,Bem József tér,0203,stele,3,999,{'icon': {'name': 'vehicle-rental/mol-bubi'}}
233,42990697,47.518845,19.081321,Széchenyi fürdő,1401,stele,10,999,{'icon': {'name': 'vehicle-rental/mol-bubi'}}
36,75859340,47.470691,19.045504,Csonka János tér,1128,stele,10,999,{'icon': {'name': 'vehicle-rental/mol-bubi'}}
25,75853903,47.471175,19.029523,Csóka utca - Bogyó utca,1121,stele,8,999,{'icon': {'name': 'vehicle-rental/mol-bubi'}}
84,42990745,47.524468,19.037193,Zsigmond tér,0212,stele,8,999,{'icon': {'name': 'vehicle-rental/mol-bubi'}}
55,42990606,47.497586,19.040916,Clark Ádám tér,0103,stele,4,999,{'icon': {'name': 'vehicle-rental/mol-bubi'}}


A simple test calculating the average number of bikes at the stations.

In [60]:
extracteddata = []
atl = 0
for mylist in data["data"]["list"]:
    if mylist["code"] != "BIKES" and mylist["spaces"] != "0":
        extracteddata.append(mylist["bikes"])
for i in range(len(extracteddata)):
    atl += extracteddata[i]
atl /= len(extracteddata)
print(f"{str(atl)[0:4]}")
    

7.24
